In [ ]:
!pip install -q wikipedia-api validators SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Percorso Eccellenza/master

/content/drive/MyDrive/Percorso Eccellenza/master


In [ ]:
import pandas as pd
from validators import url
import wikipediaapi
import requests
from bs4 import BeautifulSoup
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import json

# Get entity types

In [ ]:
# utility function to extract the kb from the uri
def get_uri_db(uri):
    if "http://dbpedia.org/resource/" in uri:
        return 'dbpedia'
    elif "yago-knowledge.org/resource/" in uri:
        return 'yago'
    elif "en.wikipedia.org/wiki" in uri:
        return 'wikipedia'

# get type from wikidata
def get_wdt(uri, uri_db):
  # if uri is from yago or dbpedia we need to find the wikidata entity using the owl:sameas property
  if uri_db == 'yago':
    sparql = SPARQLWrapper("https://yago-knowledge.org/sparql/query")
    sparql.setQuery(f"""
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    SELECT ?sameas
    WHERE
    {{
        <{uri}> owl:sameAs ?sameas
    }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    wd_uri = results['results']['bindings'][0]['sameas']['value']

  elif uri_db == 'dbpedia':
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(f"""
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        SELECT ?x
        WHERE {{ <{uri}> owl:sameAs ?x }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    for twin in results['results']['bindings']:
      if "http://www.wikidata.org/entity/" in twin['x']['value']:
        wd_uri = twin['x']['value']
        break
  # if the uri is from wikipedia we need to scrape the wikipedia page to find the wikidata URI
  elif uri_db == 'wikipedia':
    page = requests.get(uri)
    soup = BeautifulSoup(page.content, "html.parser")
    wd_uri = soup.find(id='t-wikibase').find('a')['href']

  # now we can query wikidata for the value of the "instance of" property
  sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
  q_id = wd_uri.split('/')[-1]
  sparql.setQuery(f"""
  prefix wdt: <http://www.wikidata.org/prop/direct/>
  prefix wd: <http://www.wikidata.org/entity/>
  SELECT ?type
  WHERE
  {{
    wd:{q_id} wdt:P31 ?type.
  }}
  """)
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  return results['results']['bindings'][0]['type']['value']

# get value of rdf:type when the predicted entities are from the same kb
def get_rdftype(uri, uri_db):
  if uri_db == 'dbpedia':
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(f"""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?type
        WHERE {{ <{uri}> rdf:type ?type }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for t in results['results']['bindings']:
      if "http://dbpedia.org/ontology/" in t['type']['value']:
        t_uri = t['type']['value']
        break

  elif uri_db == 'yago':
    sparql = SPARQLWrapper("https://yago-knowledge.org/sparql/query")
    sparql.setQuery(f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?type
    WHERE
    {{
        <{uri}> rdf:type ?type
    }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    t_uri = results['results']['bindings'][0]['type']['value']

  elif uri_db == 'wikipedia':
    page = requests.get(uri)
    soup = BeautifulSoup(page.content, "html.parser")
    wd_uri = soup.find(id='t-wikibase').find('a')['href']
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    q_id = wd_uri.split('/')[-1]
    sparql.setQuery(f"""
    prefix wdt: <http://www.wikidata.org/prop/direct/>
    prefix wd: <http://www.wikidata.org/entity/>
    SELECT ?type
    WHERE
    {{
      wd:{q_id} wdt:P31 ?type.
    }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    t_uri = results['results']['bindings'][0]['type']['value']

  return t_uri

In [ ]:
from tqdm import tqdm
from rdflib.term import URIRef
import time
from http.client import RemoteDisconnected

entity_vocab_inv = invert_dict(entity_vocab)

res_dict = {'Predicted URI': [], 'True URI': [], 'Predicted Type': [], 'True Type': [],'Type Match': [], 'Error Type': []}

# test_answer is the result of running the code ("sim.npy")
for i in tqdm(range(0, len(test_answer))):
    result = preds[i]
    result = result[entity_kb1_vocab]
    result = (-result).argsort()

    uri_true = entity_vocab_inv[test_answer[i]]
    uri_pred = entity_vocab_inv[result[0]]
    # if there is a mismatch
    if uri_true != uri_pred:
      # if one of the predictions is not URI save the string and save for further analysis
      if type(uri_pred) != URIRef or type(uri_true) != URIRef:
        res_dict['Predicted URI'].append(str(uri_pred))
        res_dict['True URI'].append(str(uri_true))
        res_dict['Predicted Type'].append(None)
        res_dict['True Type'].append(None)
        res_dict['Type Match'].append(None)
        res_dict['Error Type'].append('Not URIs')
      else:
        try:
          res_dict['Predicted URI'].append(str(uri_pred))
          res_dict['True URI'].append(str(uri_true))
          # get the knowledge bases
          uri_pred_db = get_uri_db(uri_pred)
          uri_true_db = get_uri_db(uri_true)
          # if they match get value of rdf:type
          if uri_pred_db == uri_true_db:
            true_type = get_rdftype(uri_true, uri_true_db)
            pred_type = get_rdftype(uri_pred, uri_pred_db)
          # else look at wikidata
          else:
            true_type = get_wdt(uri_true, uri_true_db)
            #time.sleep(1)
            pred_type = get_wdt(uri_pred, uri_pred_db)

          # entity type mismatch
          if true_type != pred_type:
            res_dict['Type Match'].append(0)
            res_dict['Error Type'].append(None)
          else:
            res_dict['Type Match'].append(1)
            res_dict['Error Type'].append(None)

          res_dict['Predicted Type'].append(pred_type)
          res_dict['True Type'].append(true_type)
          #time.sleep(1)

        # handle various connection errors
        except (IndexError, UnboundLocalError, RemoteDisconnected) as e:
            res_dict['Type Match'].append(None)
            res_dict['Error Type'].append(str(e))
            res_dict['Predicted Type'].append(None)
            res_dict['True Type'].append(None)


# Results Analysis

## Preprocessing

In [ ]:
res = pd.read_excel('results.xlsx')
print(len(res))
print(len(res[res['Error Type'].isnull()]))

2477
1480


In [ ]:
print(len(res[res['Error Type'] == 'list index out of range'])) # questi 404
res = res[res['Error Type'] != 'list index out of range']
res = res[res['Error Type'] != "local variable 'wd_uri' referenced before assignment"]

478


In [ ]:
print(len(res))
print(len(res[res['Error Type'].isnull()]))

1884
1480


In [ ]:
not_uri = res[res['Error Type'] == "Not URIs"]

In [ ]:
# look for the wikipedia pages of entities that are identified by strings instead of URIs
string_list = []

for i in not_uri['Predicted URI']:
  if not url(i):
    string_list.append(i)

for j in not_uri['True URI']:
  if not url(j):
    string_list.append(j)

print(len(not_uri) <= len(string_list))

diz = {}
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')

for s in string_list:
  page_py = wiki_wiki.page(s)
  if page_py.exists():
    categories = page_py.categories
    if "Category:All article disambiguation pages" in categories:
      print(f"Skipping {s}: disambiguation page")
    else:
      uri = page_py.fullurl
      diz[s] = uri

True
Skipping Boston Storm: disambiguation page
Skipping Big D: disambiguation page
Skipping Government Center: disambiguation page
Skipping HTA: disambiguation page
Skipping G-Men: disambiguation page
Skipping GTFC: disambiguation page
Skipping Government Center: disambiguation page
Skipping Green Army: disambiguation page
Skipping Henderson County: disambiguation page
Skipping Kickers: disambiguation page
Skipping MTFC: disambiguation page
Skipping Lilywhites: disambiguation page
Skipping Rain City: disambiguation page
Skipping Hammer (disambiguation): disambiguation page
Skipping Bale: disambiguation page
Skipping Massese: disambiguation page
Skipping HTA: disambiguation page
Skipping Rain City: disambiguation page
Skipping A6: disambiguation page
Skipping Rapid (disambiguation): disambiguation page
Skipping Panteras: disambiguation page
Skipping Hammer (disambiguation): disambiguation page
Skipping HTA: disambiguation page
Skipping Green County: disambiguation page
Skipping The Dra

In [ ]:
# map string to URI
def mapping(x):
  if x in diz:
    return diz[x]
  else:
    return x

not_uri['Predicted URI'] = not_uri['Predicted URI'].map(mapping)
not_uri['True URI'] = not_uri['True URI'].map(mapping)

In [ ]:
import time
# get entity types
for i in tqdm(range(len(not_uri))):
  pred_uri = not_uri.iloc[i, 0]
  true_uri = not_uri.iloc[i, 1]
  if not url(pred_uri) or not url(true_uri):
    continue
  else:
    uri_pred_db = get_uri_db(pred_uri)
    uri_true_db = get_uri_db(true_uri)
    try:
      true_type = get_rdftype(true_uri, uri_true_db)
      pred_type = get_rdftype(pred_uri, uri_pred_db)
      not_uri.iloc[i, 2] = pred_type
      not_uri.iloc[i, 3] = true_type
      not_uri.iloc[i, 4] = int(pred_type == true_type)
      not_uri.iloc[i, 5] = ''
    except IndexError as e:
      not_uri.iloc[i, 5] = str(e)

100%|██████████| 404/404 [17:21<00:00,  2.58s/it]


In [ ]:
not_uri.to_excel('results_not_uri.xlsx', index=False)

In [ ]:
not_uri = pd.read_excel('results_not_uri.xlsx')

In [ ]:
not_uri = not_uri[not_uri['Error Type'] != 'list index out of range'] # sempre 404

In [ ]:
not_uri['Error Type'] = 'Not URI'

In [ ]:
final_1 = not_uri[not_uri['Type Match'].notnull()]
final_2 = res[res['Type Match'].notnull()]
final = pd.concat([final_1, final_2])

In [ ]:
len(final)

2084

In [ ]:
final['Pred-True'] = final['Predicted Type'] + ' -> ' + final['True Type']

In [ ]:
final.to_excel('final_results.xlsx', index=False)

## Analysis

In [ ]:
with open("entity_mappings.json", "r") as f:
  entity_mappings = json.load(f)

In [ ]:
def remap(x):
  if x not in entity_mappings:
    return x
  else:
    return entity_mappings[x]

In [ ]:
res = pd.read_excel('final_results.xlsx')
print(len(res))
res['True Type'] = res['True Type'].map(remap)
res['Predicted Type'] = res['Predicted Type'].map(remap)
res['Type Match'] = res['True Type'] == res['Predicted Type']
mismatched = res[res['Type Match'] == False]
print(len(mismatched))

2084
1088


In [ ]:
print(f"% of correct type alignments (total): {round(res['Type Match'].mean()*100, 2)}" )
not_uri = res[res['Error Type'] == 'Not URI']
print(f"% of correct type alignments (entities different from URI): {round(not_uri['Type Match'].mean()*100, 2)}")
only_uri = res[res['Error Type'] != 'Not URI']
print(f"% of correct type alignments (only URI): {round(only_uri['Type Match'].mean()*100, 2)}")

% of correct type alignments (total): 47.79
% of correct type alignments (entities different from URI): 5.63
% of correct type alignments (only URI): 54.94


In [ ]:
mismatched['True Type'] = mismatched['True Type'].map(remap)
mismatched['Predicted Type'] = mismatched['Predicted Type'].map(remap)
mismatched['Pred-True'] = mismatched['Predicted Type'] + ' & ' + mismatched['True Type']
print(pd.DataFrame(mismatched['Pred-True'].value_counts()[:10]))

                         Pred-True
Place & Person                 178
Person & Football Club         161
Football Club & Person         152
Person & Place                 150
Career Station & Person         33
University & Person             28
Person & Career Station         26
Person & Agent                  25
Place & Football Club           24
Agent & Person                  20
